## Import datasetu

In [1]:
import deepchem as dc
import matplotlib.pyplot as plt
import polars as pl
import numpy as np


2025-12-02 20:10:12.161985: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-02 20:10:13.235623: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-02 20:10:16.066520: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


In [2]:
dc.__version__

'2.6.0.dev'

## Przegląd danych

In [ ]:
tasks, datasets, transformers = dc.molnet.load_chembl(
    featurizer=dc.feat.MACCSKeysFingerprint(),
    set='sparse',
    reload=False
)

Łączenie danych w jeden dataset

In [ ]:
train_dataset, valid_dataset, test_dataset = datasets

full_dataset = dc.data.DiskDataset.merge(datasets)

In [ ]:
# Pobranie nazw zadań (tasks)
tasks = full_dataset.get_task_names()

# Ekstrakcja danych NumPy z obiektu DeepChem Dataset
X_data = full_dataset.X
Y_data = full_dataset.y
W_data = full_dataset.w
mol_ids = full_dataset.ids

In [ ]:
# Użycie maski wag (W) do stworzenia maski logicznej
# Tam, gdzie W=0.0, wartość Y jest nieznana.
# Użyjemy tej maski, aby jawnie ustawić NaN.
unknown_mask = (W_data == 0.0)

# Kopia danych Y, gdzie nieznane wartości są ustawione na NaN
Y_data_masked = Y_data.copy()
Y_data_masked[unknown_mask] = np.nan

# 1. Tworzenie Polars DataFrame z Y_data_masked
# Nazwy kolumn bierzemy z listy tasks
df_chembl_y = pl.DataFrame(
    {task: Y_data_masked[:, i] for i, task in enumerate(tasks)}
)

# 2. Dodanie identyfikatorów
df_chembl_y = df_chembl_y.with_columns(
    pl.Series("ID", mol_ids)
)

print("\n--- Fragment Data Frame Polars (Y) ---\n")
print(df_chembl_y.head())


--- Fragment Data Frame Polars (Y) ---

shape: (5, 692)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ CHEMBL107 ┆ CHEMBL107 ┆ CHEMBL107 ┆ CHEMBL107 ┆ … ┆ CHEMBL616 ┆ CHEMBL616 ┆ CHEMBL618 ┆ ID       │
│ 5051      ┆ 5104      ┆ 5145      ┆ 5189      ┆   ┆ 4         ┆ 6         ┆ 4         ┆ ---      │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ str      │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ NaN       ┆ NaN       ┆ NaN       ┆ NaN       ┆ … ┆ NaN       ┆ NaN       ┆ NaN       ┆ CC(C)C[C │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ @H](NC(= │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ O)[C@H]( │
│           ┆           ┆         

In [ ]:
# Utworzenie Polars DataFrame z cechami (X)
# Nadanie kolumnom generycznych nazw (MACCS_0, MACCS_1, ...)
feature_names = [f"MACCS_{i}" for i in range(X_data.shape[1])]
df_chembl_x = pl.DataFrame(
    X_data,
    schema=feature_names
)
df_chembl_x = df_chembl_x.with_columns(
    pl.Series("ID", mol_ids)
)

print("\n--- Fragment Data Frame Polars (X) - Cechy MACCS ---\n")
print(df_chembl_x.head())


--- Fragment Data Frame Polars (X) - Cechy MACCS ---

shape: (5, 168)
┌─────────┬─────────┬─────────┬─────────┬───┬───────────┬───────────┬───────────┬──────────────────┐
│ MACCS_0 ┆ MACCS_1 ┆ MACCS_2 ┆ MACCS_3 ┆ … ┆ MACCS_164 ┆ MACCS_165 ┆ MACCS_166 ┆ ID               │
│ ---     ┆ ---     ┆ ---     ┆ ---     ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---              │
│ i64     ┆ i64     ┆ i64     ┆ i64     ┆   ┆ i64       ┆ i64       ┆ i64       ┆ str              │
╞═════════╪═════════╪═════════╪═════════╪═══╪═══════════╪═══════════╪═══════════╪══════════════════╡
│ 0       ┆ 0       ┆ 0       ┆ 0       ┆ … ┆ 1         ┆ 1         ┆ 0         ┆ CC(C)C[C@H](NC(= │
│         ┆         ┆         ┆         ┆   ┆           ┆           ┆           ┆ O)[C@H](CS(=O)…  │
│ 0       ┆ 0       ┆ 0       ┆ 0       ┆ … ┆ 1         ┆ 1         ┆ 0         ┆ C[C@H](\C=C(/C)\ │
│         ┆         ┆         ┆         ┆   ┆           ┆           ┆           ┆ C=C\C(=O)NO)C(…  │
│ 0       ┆ 0       

# EDA

In [ ]:
print(f"Liczba próbek (cząsteczek): {len(mol_ids)}")
print(f"Liczba zadań (bioaktywności): {len(tasks)}")
print(f"Nazwy pierwszych 5 zadań: {tasks[:5]}")

Liczba próbek (cząsteczek): 244245
Liczba zadań (bioaktywności): 691
Nazwy pierwszych 5 zadań: ['CHEMBL1075051' 'CHEMBL1075104' 'CHEMBL1075145' 'CHEMBL1075189'
 'CHEMBL1075228']


## Analiza wypełnienia

In [ ]:
# Całkowita liczba danych do przewidzenia
total_possible = Y_data.size

# Liczba znanych wartości (gdzie waga W jest różna od 0.0)
known_values_count = np.sum(W_data != 0.0)

# Procent wypełnienia
sparsity_percent = (known_values_count / total_possible) * 100

print(f"Całkowita liczba możliwych wartości Y: {total_possible}")
print(f"Liczba znanych bioaktywności (W != 0.0): {known_values_count}")
print(f"Wypełnienie zbioru danych (sparsity): {sparsity_percent:.2f}%")

Całkowita liczba możliwych wartości Y: 168773295
Liczba znanych bioaktywności (W != 0.0): 355085
Wypełnienie zbioru danych (sparsity): 0.21%
